In [3]:
import tweepy

bearer_token = "AAAAAAAAAAAAAAAAAAAAAAhrjgEAAAAAtaINtJuJDU%2F%2BCdOGL3BFon0rYzo%3DRBGMBkpmeImGMYSEnM0Sa8F2mCwZlidarulMjtMOuKqla8aWCs" # Twitter API
client = tweepy.Client(bearer_token=bearer_token) # Twitter API v2 with OAuth2.0 Version

#let's request and paginate the tweets
query = 'context:152.825047692124442624 -is:retweet lang:en'

limit = 101 #n of tweets to be requested


tweets = tweepy.Paginator(client.search_recent_tweets,
                          query=query,
                          expansions=['author_id'],
                          tweet_fields=['public_metrics'],
                          max_results=100).flatten(limit=limit)

In [4]:
#Let's create the panda dataframe
import pandas as pd

df_tweets = pd.DataFrame([tweet.data['text'],
                          tweet.data['public_metrics']['retweet_count'],
                          tweet.data['public_metrics']['like_count']
                         ] for tweet in tweets) # convert to pandas df

In [5]:
display(df_tweets)

,0,1,2
0,"I eat mostly organic, but I lo. https://t.co/N...",0,0
1,I slept the whole day 😭 and was just awake to ...,0,0
2,Potatoes for lunch making me happy!,0,0
3,@FrxRealMadrid in school me &amp; my friends s...,0,0
4,Grandbaby Cakes’ Dutch Oven Pot Roast Is the U...,0,0
...,...,...,...
96,choose my lunch,0,0
97,@tawnyuwu When I recently visited eastern part...,0,0
98,@Sanjay_IRTS Come to our state hockey world Cu...,0,0
99,Dont skip your lunch guys!,0,0


In [6]:
#Let's rename the columns
df_tweets.rename(columns={0: "Text", 1: "Retweet", 2: "Likes"})

,Text,Retweet,Likes
0,"I eat mostly organic, but I lo. https://t.co/N...",0,0
1,I slept the whole day 😭 and was just awake to ...,0,0
2,Potatoes for lunch making me happy!,0,0
3,@FrxRealMadrid in school me &amp; my friends s...,0,0
4,Grandbaby Cakes’ Dutch Oven Pot Roast Is the U...,0,0
...,...,...,...
96,choose my lunch,0,0
97,@tawnyuwu When I recently visited eastern part...,0,0
98,@Sanjay_IRTS Come to our state hockey world Cu...,0,0
99,Dont skip your lunch guys!,0,0


In [7]:
import os

path = os.getcwd()

In [6]:
_ = df_tweets.to_csv(os.path.join(path,'Dataset1.csv'))#creates a csv file containing the tweets